In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot
import plot as plot
import flux as flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.lines import Line2D

In [ ]:
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'

# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS
alpha = plot.ALPHA

# Read in Snapshots

In [ ]:
# nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

In [ ]:
# snaps = np.empty_like(nums, dtype=snapshot)
# for ii, num in enumerate(nums):
#     snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
#     # shot[ii].load_shock_variables()
#     snaps[ii].load_fluxes()

match scale axes

In [ ]:
def add_slope(ax, ls=':', color='k', alpha=0.9, lw=1,
    m1 = 2,    xmin1=10**-2,   xmax1=10**-1.5,   ymin1 =  10**-3,
    m2 = -0.1, xmin2=10**1.86, xmax2=10**2.36, ymax2 = 3.5*10**-2):

    ymax1 = ymin1 * (xmax1/xmin1)**m1
    # log y2 = log y1 + m*(logx2-logx1) = log y1 + log((x2/x1)^m)
    # y2 = y1 + (x2/x1)^m = y1 + 

    ymin2 = ymax2 / (xmax2/xmin2)**m2


    ax.plot([xmin1, xmax1], [ymin1, ymax1], ls=ls, color=color, alpha=alpha, lw=lw)
    ax.plot([xmin2, xmax2], [ymin2, ymax2], ls=ls, color=color, alpha=alpha, lw=lw)

# All-Res

In [ ]:
rnums = np.array([9, 21, 39, 54, 68])
ryears = years[1:len(rnums)+1]
rmasss = masss[1:len(rnums)+1]

snaps_lr = np.empty_like(rnums, dtype=snapshot)
snaps_mr = np.empty_like(rnums, dtype=snapshot)
# get basic snapshots for the rest
for ii, num in enumerate(rnums):
    snaps_lr[ii] = basic_snapshot(snap=num, name = ('Snap%03d_nlr' % num))
    snaps_mr[ii] = basic_snapshot(snap=num, name = ('Snap%03d_n' % num))

# assign res positions to the rest
for snaps in [snaps_lr, snaps_mr]:
    for ii, snap in enumerate(snaps):
        snap.load_fluxes(debug=False)
        
# snaps_lr, snaps_mr, snaps_hr = flux.prep_spec_snapshots()

mass_lr = np.array(['2 M$_\odot$ lr', '4 M$_\odot$ lr', '8 M$_\odot$ lr', '12 M$_\odot$ lr', '16 M$_\odot$ lr'])
mass_mr = np.array(['2 M$_\odot$ mr', '4 M$_\odot$ mr', '8 M$_\odot$ mr', '12 M$_\odot$ mr', '16 M$_\odot$ lr'])
# mass_hr = np.array(['2 M$_\odot$ hr', '4 M$_\odot$ hr', '8 M$_\odot$ hr', '12 M$_\odot$ hr',])
# print(mass_lr, mass_mr, mass_hr)

In [ ]:
print(snaps_lr[0].__dict__.keys())
print(scales)
print(f"{snaps_lr[0].ScaleFluxes.shape=}")
print(f"{snaps_lr[0].ScaleFluxes_ratio.shape=}")

In [ ]:
def plot_allres_spectra(snaps_lr=None, snaps_mr=None, snaps_hr=None, labels=None, xx=freqs/10**9, 
    row_text = ['Case A', 'Case B'], col_text = ['1000 au', '25000 au'],
    xx_text=0.99, yy_text=0.025, match_rows=False, match='rows',
    xlabel = r'Frequency, $\nu$ [GHz]', ylabel = r'Flux $S_\nu$ [mJy]',
    markers = np.array(['v', 's', 'x', '*', 'd', 'P']), colors = plot.COLORS,
    linestyles=[':', '--', '-'],
    linewidths=[2, 2, 2],
    markstyles=['x', '+', 'o']
    ):


    fig, axs = plot.figax_double(
        nrows=2, ncols=2, height=7, sharex=True)
    
    if match=='rows':
        axs[1,0].sharey(axs[0,0])
        axs[1,1].sharey(axs[0,1])
    elif match=='cols': 
        axs[0,1].sharey(axs[0,0])
        axs[1,1].sharey(axs[1,0])

    for ax in axs[1,:]:
        ax.set_xlabel(xlabel)
    for ax in axs[:,0]:
        ax.set_ylabel(ylabel)

    for ii, row in enumerate(axs):
        for jj, ax in enumerate(row):
            ax.text(xx_text, yy_text, (col_text[jj]+'\n'+row_text[ii]),
                    weight='bold', horizontalalignment='right', transform=ax.transAxes,)


    for nn, snaps in enumerate([snaps_lr, snaps_mr, snaps_hr]):  
        if snaps is not None:      
            ls = linestyles[nn]
            ms = markstyles[nn]
            lw = linewidths[nn]
            for ss, snap in enumerate(snaps):
                y00 = snap.ScaleFluxes[0,:,1]
                y01 = snap.ScaleFluxes[0,:,-1]
                y10 = snap.ScaleFluxes_ratio[:,1]
                y11 = snap.ScaleFluxes_ratio[:,-1]
                yy = np.array([y00, y01, y10, y11])
                for ii, ax in enumerate(axs.flatten()):
                    label=labels[ss] if nn==1 else None
                    ax.plot(xx, yy[ii], color=colors[ss], label=label, marker=ms, ls=ls, alpha=plot.ALPHA, lw=lw)

    # if ax0_text is not None and ax1_text is not None:
    #     axtext = np.array([ax0_text, ax0_text, ax1_text, ax1_text])
    #     for ii, ax in enumerate(axs.flatten()):
    #         ax.text(xx_text, yy_text, axtext[ii], transform=ax.transAxes,
    #                 weight='bold', horizontalalignment='right')

    return fig

res_legend_elements = [
            Line2D([0], [0], color='k', marker='x', linestyle=':', label='low-res'),
            Line2D([0], [0], color='k', marker='+', linestyle='--',  label='mid-res'),
            Line2D([0], [0], color='k', marker='o', linestyle='-',  label='high-res')]
fiducial_legend_elements = [
            Line2D([0], [0], color='k', marker=None, linestyle='--', label='low-res'),
            Line2D([0], [0], color='k', marker=None, linestyle='-',  label='fiducial'),]
            # Line2D([0], [0], color='k', marker='o', linestyle='-',  label='high-res')]

In [ ]:

# fig = plot_allres_spectra(snaps_lr, snaps_mr, snaps_hr, labels=rmasss, match='rows')
# ax = fig.axes[0]
# ax.legend(loc='upper left')
# fig.axes[1].legend(handles=res_legend_elements, bbox_to_anchor = (0.0,1.0), loc='upper left')
# fig.tight_layout()
# fig.savefig(VICO_loc+'/figures/spectra/spectra_allres.png', dpi=300)

In [ ]:
# fig = plot_allres_spectra(snaps_lr, snaps_mr, snaps_hr=None, labels=rmasss, match='cols',
#                               linestyles=['--', '-'], markstyles=[None, None])
# ax = fig.axes[0]
# ax.legend(loc='upper left')
# fig.axes[1].legend(handles=fiducial_legend_elements, bbox_to_anchor = (0.0,1.0), loc='upper left')

# for ax in [fig.axes[1], fig.axes[3]]:
#     ax.tick_params('y', labelleft=False)
# fig.subplots_adjust(wspace=0, hspace=0)
# fig.savefig(VICO_loc+'/figures/spectra/spectra_allres_allcase_matchy.png', dpi=300)

In [ ]:
fig = plot_allres_spectra(snaps_lr, snaps_mr, snaps_hr=None, labels=rmasss, match=None,
                              linestyles=['--', '-'], markstyles=[None, None])
axs = fig.axes
add_slope(axs[0], ymin1=3*10**-4, ymax2=3*10**0, lw=2)
add_slope(axs[1], ymin1=7*10**-2, ymax2=7*10**1, lw=2)
add_slope(axs[2], ymin1=3*10**-4, ymax2=2*10*-2, lw=2)
add_slope(axs[3], ymin1=2*10**-2, ymax2=1*10**-1, lw=2)


leg1 = axs[0].legend(loc='upper left')
fig.axes[1].legend(handles=fiducial_legend_elements, bbox_to_anchor = (0.0,1.0), loc='upper left')

fig.subplots_adjust(wspace=0.1, hspace=0)
fig.savefig(VICO_loc+'/figures/spectra/spectra_allres.png', dpi=200)